In [ ]:
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import numpy as np
from long_covid.colors import flatuicolors
from long_covid import styling

In [ ]:
TESTS = pd.read_feather('../data/01_raw/tests.feather')
VACCINATION = pd.read_feather('../data/01_raw/vaccinations.feather')
METADATA = pd.merge(TESTS, VACCINATION, on='user_id')
WEEKLY_DEVIATIONS = pd.read_feather('../data/03_derived/weekly_vital_deviations_per_user.feather')
COHORTS = pd.read_feather('../data/03_derived/user_cohorts.feather')

In [ ]:
def figure_breakthrough_timing():

    f, ax = plt.subplots(figsize=(4,3))
    
    vaccinated_users = COHORTS[COHORTS.vaccinated].user_id.values
    valid_users = WEEKLY_DEVIATIONS[WEEKLY_DEVIATIONS.userid.isin(vaccinated_users)].userid.unique()
    
    temp = METADATA[METADATA.user_id.isin(valid_users)].copy()
    temp['dt'] = temp.test_date - temp.third_dose
    temp.loc[temp.dt.isna(), 'dt'] = temp[temp.dt.isna()].test_date - temp[temp.dt.isna()].second_dose
    
    x = np.array([t.days // 30 for t  in temp.dt])

    print('Share of breakthrough cases in month after vaccination:', (x == 0).sum() / len(x))
    
    ax.hist(x, bins=np.arange(-0.4, 11.5, 1), width=0.8, color=flatuicolors.midnightblue)
    styling.hide_and_move_axis(ax)
    ax.set_xlabel('Months since vaccination')
    ax.set_ylabel('Number of recorded\nbreakthrough infections')
    
    plt.tight_layout()
    plt.savefig('../output/si_figure4_time_between_vaccination_and_infection.jpg', dpi=400)

In [ ]:
figure_breakthrough_timing()